In [18]:
#!pip install openfoodfacts

In [19]:
import pandas as pd
import openfoodfacts
from collections import Counter

# Read data from openfoodfacts and create df

In [20]:
ids = [737628064502,3017620422003,5449000131805,3175680011534,8000500310427,3228857000166,3229820782560,5410188031072,5010477348630,3068320114453,3088543506255,3033490506629,7622210476104,5000112611878,3228021170022,5411188119098,3073781115345,3252210390014,20724696,8076809513753,87157239,7622300441937,5053990156009,20916435]
ids = [str(x) for x in ids]

In [21]:
categories = ["id", "categories_hierarchy", "generic_name", "nutriscore_score", "quantity", "origins", "Allergens"]

In [22]:
df = pd.DataFrame(columns=categories)

In [23]:
for product_id in ids:
    product = openfoodfacts.products.get_product(product_id)
    values = list( map(product.get("product").get, categories) )
    df.loc[len(df)] = values
df.head()

,id,categories_hierarchy,generic_name,nutriscore_score,quantity,origins,Allergens
0,737628064502,"[en:plant-based-foods-and-beverages, en:plant-...",Rice Noodles,4,155 g,Thailand,None
1,3017620422003,"[en:Petit-déjeuners, en:Produits à tartiner, e...",None,26,400 g,,None
2,5449000131805,"[en:beverages, en:carbonated-drinks, en:artifi...","Boisson rafraîchissante aux extraits végétaux,...",1,330 ml,,None
3,3175680011534,"[en:snacks, en:breakfasts, en:sweet-snacks, en...",None,9,230 g,"France,European Union,Non European Union",None
4,8000500310427,"[en:snacks, en:sweet-snacks, en:biscuits-and-c...",None,25,304g,,None


In [24]:
# Unnecessary work 

#get all categories
'''categ=openfoodfacts.facets.get_categories()
type(categ[0]), categ[0]
all_categories = list(map(lambda d: d.get('id', 'default value'), categ))'''


#split and add categories as column
'''%%time
df_test = pd.DataFrame([pd.Series(x) for x in df.categories_hierarchy])
df_test.columns = [f'category_{x+1}' for x in df_test.columns]
df_res = pd.concat([df, df_test], axis=1)'''

"%%time\ndf_test = pd.DataFrame([pd.Series(x) for x in df.categories_hierarchy])\ndf_test.columns = [f'category_{x+1}' for x in df_test.columns]\ndf_res = pd.concat([df, df_test], axis=1)"

# Create new DF with splitted categories and counter 

In [25]:
category_counter = Counter(df['categories_hierarchy'].explode())

In [26]:
category_counter_df= pd.DataFrame.from_dict(category_counter, orient='index').reset_index().rename(columns={"index": "category", 0: "amount"})

In [27]:
category_counter_df.head()

,category,amount
0,en:plant-based-foods-and-beverages,8
1,en:plant-based-foods,7
2,en:cereals-and-potatoes,4
3,en:cereals-and-their-products,3
4,en:pastas,1


In [28]:
# Unnecessary work 

#get df with categories that occure once
'''mask_for_one_occurance = (category_counter_df.amount == 1)
df.categories_hierarchy.apply(lambda x: any([k in x for k in category_counter_df[mask_for_one_occurance].category.to_list()]))'''

'mask_for_one_occurance = (category_counter_df.amount == 1)\ndf.categories_hierarchy.apply(lambda x: any([k in x for k in category_counter_df[mask_for_one_occurance].category.to_list()]))'

# Find and Add min and max to new DF

In [29]:
aggr_df = df.explode("categories_hierarchy").groupby("categories_hierarchy").agg({"nutriscore_score": ['min', 'max']}).xs('nutriscore_score', axis=1, drop_level=True).reset_index()


In [30]:
result_df = aggr_df.rename(columns={"categories_hierarchy": "category", "min": "minimum", "max": "maximum"}).merge(category_counter_df, on="category")

In [31]:
result_df=result_df.rename(columns={"category": "generic_name", "minimum":"min_nutr_score", "maximum":"max_nutr_score"})

In [32]:
result_df.to_csv("food_data.csv")
